In [1]:
import random,timeit
from sklearn import preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from copy import deepcopy as dc
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection as ms

In [2]:
def random_search(n,dim):
    gens=[[0 if g != j else 1 for g in range(n)] for j in range(dim)]
    return gens



In [3]:
def evaluate(train_d,train_l,gen):
        mask=np.array(gen) > 0
        al_data=np.array([al[mask] for al in train_d])
        kf = ms.KFold(n_splits=4)
        s = 0
        for tr_ix,te_ix in kf.split(al_data):
            s+= RandomForestClassifier(n_estimators=25).fit(al_data[tr_ix],train_l[tr_ix]).score(al_data[te_ix],train_l[te_ix])#.predict(al_test_data)
        s/=4
        return s

In [4]:
def bees_optimization(bee,binary,i):
    binary=list(binary)
    j=random.randint(0,len(binary)-1)
    k=random.randint(0,len(binary)-1)
    while k==i:
        k=random.randint(0,len(binary)-1)
    fit=binary[j]+random.uniform(-1,1)*(binary[j]-binary[k])
    for x in range(bee):
        y=random.randint(0,len(binary)-1)
        while y==i:
            y=random.randint(0,len(binary)-1)
        r=random.uniform(0,1)
        if r<=fit:
            binary[y]=1
        
    return binary

In [5]:
def BABCO(train_d,train_l,n=10,max_iter=25,employee_percent=0.5,max_limit=5):
    """
    input:{ Eval_Func: Evaluate_Function, type is class
            n: Number of population, default=20
            max_iter: Number of max iteration, default=300
            }
    output:{
            Best position: type list(int) [1,0,0,1,.....]
            Nunber of 1s in best position: type int [0,1,1,0,1] → 3
            }
    """
    employed_bees = int(round(n*employee_percent))
    onlooker_bees = n - employed_bees       

    dim=len(train_d[0])
    global_best=float("-inf")
    global_position=tuple([0]*dim)
    gens_dict = {}
    limit=[0]*dim
    gens=random_search(dim,dim)
    for gen in gens:
        if tuple(gen) in gens_dict:
            score = gens_dict[tuple(gen)]
        else:
            score=evaluate(train_d,train_l,gen)
            gens_dict[tuple(gen)]=score
        if score > global_best:
            global_best=score
            global_position=dc(gen)
            
            
    for it in range(max_iter):
        for i in range(employed_bees):
            for i,x in enumerate(gens):
                gen=bees_optimization(employed_bees,x,i)
                if tuple(gen) in gens_dict:
                    score = gens_dict[tuple(gen)]
                else:
                    score=evaluate(train_d,train_l,gen)
                    gens_dict[tuple(gen)]=score

                if score > gens_dict[tuple(gens[i])]:
                    limit[i]=0
                    gens[i]= gen
                else:
                    limit[i]+=1

                if score > global_best:
                    global_best=score
                    global_position=dc(gen)

                if limit[i]>=max_limit:
                    gens[i]=[0 if g != i else 1 for g in range(dim)]
    
        for i in range(onlooker_bees):
            for i,x in enumerate(gens):
                gen=bees_optimization(employed_bees,x,i)
                if tuple(gen) in gens_dict:
                    score = gens_dict[tuple(gen)]
                else:
                    score=evaluate(train_d,train_l,gen)
                    gens_dict[tuple(gen)]=score

                if score > gens_dict[tuple(gens[i])]:
                    limit[i]=0
                    gens[i]= gen
                else:
                    limit[i]+=1

                if score > global_best:
                    global_best=score
                    global_position=dc(gen)

                if limit[i]>=max_limit:
                    gens[i]=[0 if g != i else 1 for g in range(dim)]

                
    return global_position,global_position.count(1)


In [6]:
def test_score(gen,tr_x,tr_y,te_x,te_y):
    mask=np.array(gen) == 1
    al_data=np.array(tr_x[:,mask])
    al_test_data=np.array(te_x[:,mask])
    return np.mean([RandomForestClassifier(n_estimators=25).fit(al_data,tr_y).score(al_test_data,te_y) for i in range(5)])

In [7]:
with open("/home/shubbham28/Downloads/bc/wdbc1.csv") as f:
    x=np.array([[float(d) for d  in data.split(',')] for data in f.read().splitlines()])
with open("/home/shubbham28/Downloads/bc/wdbc2.csv") as f:
    y=np.array([float(data) for data in f.read().splitlines()])
lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(y)
train_d, test_d, train_l, test_l = train_test_split(x, training_scores_encoded, test_size=0.25)

In [8]:
k=[1 for r in range(len(x[0]))]
print test_score(k,train_d,train_l,test_d,test_l)
fattr=0
ftest=0.0
flist=[0 for r in range(len(x[0]))]
final_list=[0 for r in range(len(x[0]))]
start=timeit.default_timer()
for i in range(20):
    g,l=BABCO(train_d,train_l,n=10,max_iter=25,employee_percent=0.5,max_limit=5)
    fattr+=l
    test=test_score(g,train_d,train_l,test_d,test_l)
    ftest+=test
    for j in range(len(flist)):
        if g[j]==1:
            flist[j]+=1
    print("{0}  {1}  {2}  {3:.6f}".format(i+1,"".join(map(str,g)),l,test))
fattr=fattr/20
ftest=ftest/20
end=timeit.default_timer()
time=end-start
final=np.argsort(flist)[::-1][:fattr]
for i in range(len(final)):
    final_list[final[i]]=1
print("{0}  {1}   {2}   {3:.6f}    {4:.4f}".format("Final: ","".join(map(str,final_list)),fattr,ftest,time))

0.9622377622377621
1  010011010100100001010111101011  15  0.966434
2  111101111010111100100101001011  19  0.959441
3  111001111111110000100111011011  20  0.960839
4  110000001011000110100001101011  13  0.962238
5  011100011111001000110110111000  16  0.967832
6  110110111011110011110101111010  21  0.973427
7  011011001101010011100001111111  18  0.972028
8  110010011111110110110101001010  18  0.958042
9  110011011111000000000010101101  14  0.965035
10  011110010000010000000100001110  10  0.962238
11  011100011000010101001000001000  10  0.962238
12  110000111111111100010101000011  17  0.960839
13  010011110000010110111100011110  16  0.977622
14  010011010000010000001101001000  9  0.959441
15  010010000100000000100101110000  8  0.976224
16  101011000101101000000101111010  14  0.965035
17  010110010010010111000101101001  14  0.963636
18  010010101010000110001101001111  14  0.979021
19  010111011011101010010001001000  14  0.948252
20  010100010100111011010101100010  14  0.960839
Final:   010

In [7]:
with open("/home/shubbham28/Downloads/Arrhythmia/heart.csv") as f:
    x=np.array([[float(d) for d  in data.split(',')] for data in f.read().splitlines()])
with open("/home/shubbham28/Downloads/Arrhythmia/heart_output.csv") as f:
    y=np.array([float(data) for data in f.read().splitlines()])
lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(y)
train_d, test_d, train_l, test_l = train_test_split(x, training_scores_encoded, test_size=0.25)

In [8]:
k=[1 for r in range(len(x[0]))]
print test_score(k,train_d,train_l,test_d,test_l)
fattr=0
ftest=0.0
flist=[0 for r in range(len(x[0]))]
final_list=[0 for r in range(len(x[0]))]
start=timeit.default_timer()
for i in range(20):
    g,l=BABCO(train_d,train_l,n=10,max_iter=25,employee_percent=0.5,max_limit=5)
    fattr+=l
    test=test_score(g,train_d,train_l,test_d,test_l)
    ftest+=test
    for j in range(len(flist)):
        if g[j]==1:
            flist[j]+=1
    print("{0}  {1}  {2}  {3:.6f}".format(i+1,"".join(map(str,g)),l,test))
fattr=fattr/20
ftest=ftest/20
end=timeit.default_timer()
time=end-start
print flist,fattr
final=np.argsort(flist)[::-1][:fattr]
print final
for i in range(len(final)):
    final_list[final[i]]=1
print("{0}  {1}   {2}   {3:.6f}    {4:.4f}".format("Final: ","".join(map(str,final_list)),fattr,ftest,time))

0.7999999999999998
1  1111111111111  13  0.802941
2  1111010000111  8  0.841176
3  1011110010110  8  0.755882
4  1111111010111  11  0.838235
5  1111010011111  10  0.805882
6  0111011011110  9  0.767647
7  1110010010111  8  0.832353
8  1110011010110  8  0.808824
9  1111011110111  11  0.797059
10  1110011111111  11  0.817647
11  1111111010110  10  0.791176
12  1111001110111  10  0.817647
13  1111011011110  10  0.750000
14  1011111110111  11  0.788235
15  1110010111111  10  0.788235
16  1111011110111  11  0.826471
17  1110111011110  10  0.758824
18  1111011111111  12  0.814706
19  1111101110111  11  0.826471
20  1111011111110  11  0.770588
[19, 18, 20, 15, 7, 18, 15, 10, 19, 9, 20, 20, 13] 10
[11 10  2  8  0  5  1  6  3 12]
Final:   1111011010111   10   0.800000    2338.7517
